<a href="https://colab.research.google.com/github/Prudhvi0520/DataminingAssign1/blob/main/model_2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Colab Notebooks/final/final/images.zip", 'r')
zip_ref.extractall("/content/images")
zip_ref.close()
!pip install livelossplot
import os 
import glob
import numpy as np
from sklearn.preprocessing import LabelEncoder 
import tensorflow as tf
from livelossplot.tf_keras import PlotLossesCallback
import cv2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# understanding the dataset structure
data_dir = '../images/images/'
training_path = '/content/images/images/images/train/'
validation_path = '/content/images/images/images/validation/'


# Training classes
print("--------- Training set description --------------")
print("S.No.\tSample Count\tClass\t")
# Printing distribution of each class in training set
for (i,expression) in enumerate(os.listdir(training_path)):
    print(str(i+1)+ "\t"+ str(len(os.listdir(training_path + expression))) + "\t\t" + expression )
    

# Validation classes
print("\n--------- Validation set description --------------")
print("S.No.\tSample Count\tClass\t")
# Printing distribution of each class in training set
for (i,expression) in enumerate(os.listdir(validation_path)):
    print(str(i+1)+ "\t"+ str(len(os.listdir(validation_path + expression))) + "\t\t" + expression )


In [ ]:
def create_dataset(main_path):
    label_encoder = LabelEncoder()

    df = {"img":[],"img_class":[]}
    for class_names in os.listdir(main_path):
        for img_path in glob.glob(f"{main_path}/{class_names}/*"):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (48, 48))
            img = img[..., np.newaxis] / 255.0         
            df["img"].append(img)
            df["img_class"].append(class_names)
    df["img"] = np.array(df["img"], dtype='float32')
    df["img_class"] = label_encoder.fit_transform(df["img_class"])
    df["img_class"] = tf.keras.utils.to_categorical(df["img_class"])
    return df["img"],df["img_class"]
    
training_dataset, training_labels = create_dataset(training_path[:len(training_path)-1])
validation_dataset, validation_labels  = create_dataset(validation_path[:len(validation_path)-1])



In [ ]:
# Defining constants 
BATCH_SIZE = 64
img_channel = 3
LEARNING_RATE = 0.0001

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='reflect'
)


<h3>Building Model</h3>

In [ ]:

from tensorflow.keras.layers import Dense,  Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam


model = Sequential()

# 1 - Convolution
model.add(Conv2D(32, (3, 3),  input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))


# 2nd Convolution layer
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))


# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(7, activation='softmax'))

opt = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="accuracy",patience=3,mode="auto")
checkpoint = tf.keras.callbacks.ModelCheckpoint("best_weights.h5",verbose=1,save_best_only=True,save_weights_only = True)

In [ ]:
history = model.fit(
    data_augmentation.flow(training_dataset, training_labels, batch_size=BATCH_SIZE),    
    epochs=100,
    callbacks=[PlotLossesCallback(),checkpoint, early_stopping],
    validation_data=(validation_dataset,validation_labels))


In [ ]:
# Save Model
model.save("FacialExpressionModel.h5")